# Dual Grid Construction

A dual grid is a secondary grid built from a grid, by constructing new faces over the original grids nodes. One property of this means that if you take the dual grid of a dual grid, the orignal grid will be constructed. However, this will only work properly if the grid is not partial. UXarray allows the constructing of this dual grid on all three of our data structures using the following methods:

* `Grid.get_dual()`
* `UxDataArray.get_dual()`
* `UxDataset.get_dual()`

In [ ]:
import uxarray as ux
import warnings
import cartopy.crs as ccrs

warnings.filterwarnings("ignore")

In [ ]:
file = "../../test/meshfiles/mpas/QU/mesh.QU.1920km.151026.nc"

uxds = ux.open_dataset(file, file)

## Computing the Dual Grid

When computing the dual of a grid, the original grid is typically referred to as the "Primal" grid. The corner nodes of the Primal grid become the face centers of the Dual grid, and the face centers of the Primal grid become the corner nodes of the Dual grid. This means that variables that were originally face-centered on the Primal grid will be node-centered on the Dual grid, and vice versa. Using UXarray we can construct the dual mesh using `grid.compute_dual()`, which returns a new `Grid` object.

In [ ]:
grid = uxds.uxgrid

In [ ]:
dual = grid.get_dual()

In [ ]:
(
    grid.plot(title="Primal Grid", backend="bokeh", projection=ccrs.Orthographic())
    + dual.plot(title="Dual Grid", backend="bokeh", projection=ccrs.Orthographic())
)

In [ ]:
(
    grid.plot(backend="bokeh", projection=ccrs.Orthographic(), color="red")
    * dual.plot(backend="bokeh", projection=ccrs.Orthographic(), color="blue")
).opts(title="Primal (Red) & Dual (Blue) Grids")

## Computing the Dual Grid with Data

We can also take a dual mesh of a `UxDataArray`. The concept for constructing the `Grid` remains the same, and what is constructed will be identical. The difference is the data stored inside the `UxDataArray` will be transfered with the dual mesh. The key differences is the location that the data is stored. The data transfer process works as follows:

* Face centered data becomes node centered, as each face becomes a node in the dual mesh.
* Node centered data becomes face centered, as each node becomes a face in the dual mesh.
* Edge centered data remains unchanged, as the edge centers will remain in the same place, despite the edges themselves being different.

### Face Centered Data

When constructing the dual mesh paired with a face centered variable, the data becomes node centered. We can then plot this using `topological_mean` to get the dual data to the faces for proper visualization comparisions.

In [ ]:
uxds_dual_face = uxds["latCell"].get_dual()

In [ ]:
(
    uxds["latCell"].plot.polygons(
        rasterize=True,
        backend="matplotlib",
        title="Face centered data on Primal Mesh",
        cmap=ux.cmaps.sequential_green_blue,
        projection=ccrs.Orthographic(),
    )
    + uxds_dual_face.topological_mean(destination="face").plot.polygons(
        rasterize=True,
        backend="matplotlib",
        title="Node Centered Data on Dual Mesh",
        cmap=ux.cmaps.sequential_green_blue,
        projection=ccrs.Orthographic(),
    )
).cols(1).opts(fig_size=125)

### Node Centered Data

When constructing the dual mesh paired with a node centered variable, the data becomes face centered. A benefit of computing the dual of a node centered variable is that it allows us to visualize the data as polygons.

In [ ]:
uxds_dual_node = uxds["latVertex"].get_dual()

In [ ]:
(
    uxds["latVertex"]
    .topological_mean(destination="face")
    .plot.polygons(
        rasterize=True,
        backend="matplotlib",
        title="Face centered data on Primal Mesh",
        cmap=ux.cmaps.sequential_green_blue,
        projection=ccrs.Orthographic(),
    )
    + uxds_dual_node.plot.polygons(
        rasterize=True,
        backend="matplotlib",
        title="Node Centered Data on Dual Mesh",
        cmap=ux.cmaps.sequential_green_blue,
        projection=ccrs.Orthographic(),
    )
).cols(1).opts(fig_size=125)

### Edge Centered Data

When constructing the dual mesh paired with an edge centered variable, the data will stay edge centered. However, a plotting example cannot be shown, as the `topological_mean` needed to visualize edge centered data is not currently implemented for edge centered data.

### UxDataset

We can also construct a dual mesh from an entire dataset, which will convert the whole `UxDataset` to its dual mesh form. Below we can see the dataset before the dual mesh is constructed.

In [ ]:
uxds

Now we can construct the dual and see the new dataset that is returned.

In [ ]:
uxds_dual = uxds.get_dual()

In [ ]:
uxds_dual

As you can see, transforms the whole dataset. We can now take any variable and plot it, as shown below.

In [ ]:
uxds_dual["latVertex"].plot.polygons(
    rasterize=True,
    backend="matplotlib",
    title="latVertex from UxDataset dual mesh",
    cmap=ux.cmaps.sequential_green_blue,
    projection=ccrs.Orthographic(),
).opts(fig_size=120)